============================================================================
SECTION 1: SETUP AND IMPORTS
============================================================================

In [ ]:
!pip install -q kaggle
!pip install -q imbalanced-learn
!pip install -q plotly

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import json
from pathlib import Path
import zipfile
import shutil

In [ ]:
# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, MobileNetV2, EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
# Machine Learning & Data Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from imblearn.over_sampling import SMOTE

In [ ]:
# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

============================================================================
SECTION 2: KAGGLE DATASET DOWNLOAD
============================================================================

In [ ]:
import os, zipfile
import kagglehub

In [ ]:
path = kagglehub.dataset_download("nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone")
print("KaggleHub dataset base path:", path)

In [ ]:
def find_kidney_dataset_folder(base_path):
    for root, dirs, files in os.walk(base_path):
        if all(cls in dirs for cls in ['Cyst', 'Normal', 'Stone', 'Tumor']):
            print(f"✅ Found dataset folder at: {root}")
            return root
    print("❌ Could not find class folders automatically.")
    return None

In [ ]:
dataset_path = find_kidney_dataset_folder(path)

In [ ]:
if dataset_path is None:
    raise FileNotFoundError("Dataset folders (Cyst/Normal/Stone/Tumor) not found.")
else:
    expected_classes = ['Cyst', 'Normal', 'Stone', 'Tumor']
    for cls in expected_classes:
        print(f"{cls} folder exists:", os.path.exists(os.path.join(dataset_path, cls)))

============================================================================
SECTION 3: DATA LOADING AND PREPROCESSING
============================================================================

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
class DataLoader:
    def __init__(self, base_path, img_size=(128, 128)):
        self.base_path = base_path
        self.img_size = img_size
        self.classes = ['Cyst', 'Normal', 'Stone', 'Tumor']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def load_images(self):
        """Load and preprocess images from dataset folders"""
        images = []
        labels = []

        for class_name in self.classes:
            class_path = os.path.join(self.base_path, class_name)
            if not os.path.exists(class_path):
                print(f"Warning: Path {class_path} not found")
                continue

            image_files = [f for f in os.listdir(class_path)
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            print(f"Loading {len(image_files)} images from {class_name}...")

            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                try:
                    img = Image.open(img_path).convert('L')      # grayscale
                    img = img.resize(self.img_size)               # 128×128
                    img_array = np.array(img, dtype=np.float32)
                    img_array = (img_array - 127.5) / 127.5       # normalize to [-1, 1]
                    images.append(img_array)
                    labels.append(self.class_to_idx[class_name])
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
                    continue

        images = np.expand_dims(np.array(images), axis=-1)
        labels = np.array(labels)

        print(f"\nTotal images loaded: {len(images)}")
        print(f"Image shape: {images.shape}")
        print(f"Labels shape: {labels.shape}")
        return images, labels

    def get_class_distribution(self, labels):
        unique, counts = np.unique(labels, return_counts=True)
        return dict(zip([self.classes[i] for i in unique], counts))

In [ ]:
data_loader = DataLoader(dataset_path)
X_original, y_original = data_loader.load_images()

In [ ]:
class_dist = data_loader.get_class_distribution(y_original)
print("\nOriginal Class Distribution:")
for cls, count in class_dist.items():
    print(f"  {cls}: {count}")
# Expected: Cyst: 3709 | Normal: 5077 | Stone: 1377 | Tumor: 2283 | Total: 12446

============================================================================
SECTION 4: CLASS IMBALANCE VISUALIZATION
============================================================================

In [ ]:
def visualize_class_distribution(labels, classes, title="Class Distribution"):
    unique, counts = np.unique(labels, return_counts=True)
    class_names = [classes[i] for i in unique]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

    ax1.bar(class_names, counts, color=colors)
    ax1.set_xlabel('Class', fontsize=12)
    ax1.set_ylabel('Number of Images', fontsize=12)
    ax1.set_title(title, fontsize=14)
    ax1.grid(axis='y', alpha=0.3)
    for i, (name, count) in enumerate(zip(class_names, counts)):
        ax1.text(i, count, str(count), ha='center', va='bottom')

    ax2.pie(counts, labels=class_names, autopct='%1.1f%%', colors=colors, startangle=90)
    ax2.set_title('Class Proportion', fontsize=14)

    plt.tight_layout()
    plt.savefig('class_distribution_original.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
visualize_class_distribution(y_original, data_loader.classes,
                             "Original Dataset Class Distribution")

============================================================================
SECTION 5: DIMENSIONALITY REDUCTION (t-SNE & PCA)
============================================================================

In [ ]:
def visualize_tsne_pca(X, y, classes, title_prefix="Original"):
    X_flat = X.reshape(X.shape[0], -1)

    if len(X_flat) > 5000:
        indices = np.random.choice(len(X_flat), 5000, replace=False)
        X_sample = X_flat[indices]
        y_sample = y[indices]
    else:
        X_sample = X_flat
        y_sample = y

    print(f"Computing t-SNE for {title_prefix} data...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    X_tsne = tsne.fit_transform(X_sample)

    print(f"Computing PCA for {title_prefix} data...")
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_sample)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

    for idx, class_name in enumerate(classes):
        mask = y_sample == idx
        ax1.scatter(X_tsne[mask, 0], X_tsne[mask, 1],
                    c=colors[idx], label=class_name, alpha=0.6, s=20)
    ax1.set_title(f't-SNE Visualization - {title_prefix}', fontsize=14)
    ax1.set_xlabel('Component 1')
    ax1.set_ylabel('Component 2')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    for idx, class_name in enumerate(classes):
        mask = y_sample == idx
        ax2.scatter(X_pca[mask, 0], X_pca[mask, 1],
                    c=colors[idx], label=class_name, alpha=0.6, s=20)
    ax2.set_title(f'PCA Visualization - {title_prefix}', fontsize=14)
    ax2.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    ax2.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'tsne_pca_{title_prefix.lower().replace(" ", "_")}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
visualize_tsne_pca(X_original, y_original, data_loader.classes, "Original Dataset")

============================================================================
SECTION 6: SMOTE AUGMENTATION
============================================================================

In [ ]:
def apply_smote(X, y, target_samples):
    print("\n" + "="*60)
    print("APPLYING SMOTE AUGMENTATION")
    print("="*60)

    X_flat = X.reshape(X.shape[0], -1)

    unique, counts = np.unique(y, return_counts=True)
    max_samples = max(counts)

    # Ensure target samples meet minimum class size requirement
    if target_samples < max_samples:
        print(f"Warning: target_samples ({target_samples}) < largest class ({max_samples}).")
        print(f"Setting target_samples to {max_samples}.")
        target_samples = max_samples

    sampling_strategy = {i: target_samples for i in unique}

    print(f"\nTarget samples per class: {target_samples}")
    print("Applying SMOTE...")

    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42, k_neighbors=5)
    X_smote_flat, y_smote = smote.fit_resample(X_flat, y)

    X_smote = X_smote_flat.reshape(-1, 128, 128, 1)
    X_smote = np.clip(X_smote, -1, 1)

    print(f"\nSMOTE Augmentation Complete!")
    print(f"Original dataset size: {len(X)}")
    print(f"SMOTE dataset size: {len(X_smote)}")

    unique, counts = np.unique(y_smote, return_counts=True)
    print("\nSMOTE-Augmented Class Distribution:")
    for idx, count in zip(unique, counts):
        print(f"  {data_loader.classes[idx]}: {count}")

    return X_smote, y_smote

In [ ]:
X_smote, y_smote = apply_smote(X_original, y_original, target_samples=4000)

In [ ]:
visualize_class_distribution(y_smote, data_loader.classes,
                             "SMOTE-Augmented Class Distribution")
visualize_tsne_pca(X_smote, y_smote, data_loader.classes, "SMOTE Augmented")

============================================================================
SECTION 7: ACGAN IMPLEMENTATION
============================================================================

In [ ]:
class ACGAN:
    def __init__(self, img_shape=(128, 128, 1), num_classes=4, latent_dim=100):
        self.img_shape = img_shape
        self.num_classes = num_classes
        self.latent_dim = latent_dim
        self.optimizer = keras.optimizers.Adam(0.0002, 0.5)

        # Build and compile discriminator first
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'sparse_categorical_crossentropy'],
            optimizer=self.optimizer,
            metrics=['accuracy', 'accuracy']
        )

        # Build generator 
        self.generator = self.build_generator()

        # Build combined model with frozen discriminator
        noise = layers.Input(shape=(self.latent_dim,))
        label = layers.Input(shape=(1,))
        img = self.generator([noise, label])

        self.discriminator.trainable = False  
        valid, target_label = self.discriminator(img)

        self.combined = models.Model([noise, label], [valid, target_label])
        self.combined.compile(
            loss=['binary_crossentropy', 'sparse_categorical_crossentropy'],
            optimizer=self.optimizer
        )

    def build_generator(self):
        """Build generator network — architecture unchanged from paper"""
        noise = layers.Input(shape=(self.latent_dim,))
        label = layers.Input(shape=(1,), dtype='int32')

        label_embedding = layers.Embedding(self.num_classes, self.latent_dim)(label)
        label_embedding = layers.Flatten()(label_embedding)
        model_input = layers.Multiply()([noise, label_embedding])

        x = layers.Dense(16 * 16 * 128)(model_input)
        x = layers.LeakyReLU(negative_slope=0.2)(x)
        x = layers.Reshape((16, 16, 128))(x)

        x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)

        x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)

        x = layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)

        img = layers.Conv2DTranspose(1, kernel_size=4, strides=1, padding='same',
                                     activation='tanh')(x)

        model = models.Model([noise, label], img, name='generator')
        return model

    def build_discriminator(self):
        """Build discriminator network — architecture unchanged from paper"""
        img = layers.Input(shape=self.img_shape)

        x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(img)
        x = layers.LeakyReLU(negative_slope=0.2)(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
        x = layers.LeakyReLU(negative_slope=0.2)(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Flatten()(x)

        validity = layers.Dense(1, activation='sigmoid', name='validity')(x)
        label    = layers.Dense(self.num_classes, activation='softmax', name='label')(x)

        model = models.Model(img, [validity, label], name='discriminator')
        return model

    def train(self, X_train, y_train, epochs=100, batch_size=32, save_interval=10):
        """Train ACGAN
        [paper p.12: "training occurs over 100 epochs"] → epochs=100
        batch_size not stated in paper → keep 32
        """
        print("\n" + "="*60)
        print("TRAINING ACGAN")
        print("="*60)

        valid = np.ones((batch_size, 1))
        fake  = np.zeros((batch_size, 1))

        history = {'d_loss': [], 'g_loss': []}

        for epoch in range(epochs):
            # Train Discriminator 
            idx    = np.random.randint(0, X_train.shape[0], batch_size)
            imgs   = X_train[idx]
            labels = y_train[idx]

            noise          = np.random.normal(0, 1, (batch_size, self.latent_dim))
            sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
            gen_imgs       = self.generator.predict([noise, sampled_labels], verbose=0)

            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs,
                                                             [fake, sampled_labels.flatten()])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator 
            noise          = np.random.normal(0, 1, (batch_size, self.latent_dim))
            sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)

            g_loss = self.combined.train_on_batch([noise, sampled_labels],
                                                   [valid, sampled_labels.flatten()])

            history['d_loss'].append(d_loss[0])
            history['g_loss'].append(g_loss[0])

            if epoch % save_interval == 0:
                print(f"Epoch {epoch}/{epochs} - D Loss: {d_loss[0]:.4f}, "
                      f"D Acc: {100*d_loss[3]:.2f}%, G Loss: {g_loss[0]:.4f}")

        print("\nACGAN Training Complete!")
        return history

    def generate_images_topup(self, class_counts, target_per_class=7000):
        """
        [FIX-4] Generate exactly enough synthetic images so that each class
        reaches target_per_class when combined with original images.

        Paper p.12: "increasing the total number of images per class up to 7,000,
        which results in a balanced dataset of 28,000 unique images"

        class_counts: dict {class_idx: original_count}
        target_per_class: 7000

        Returns: (X_synthetic, y_synthetic)
          Cyst (0):   7000 - 3709 = 3291 synthetic images
          Normal (1): 7000 - 5077 = 1923 synthetic images
          Stone (2):  7000 - 1377 = 5623 synthetic images
          Tumor (3):  7000 - 2283 = 4717 synthetic images
          Total synthetic: 15,554
          Total combined:  28,000
        """
        print(f"\nGenerating per-class top-up synthetic images (target: {target_per_class}/class)...")

        generated_images = []
        generated_labels = []

        for class_idx in range(self.num_classes):
            orig_count = class_counts[class_idx]
            needed     = target_per_class - orig_count

            if needed <= 0:
                print(f"  Class {class_idx}: already has {orig_count} images, no generation needed.")
                continue

            print(f"  Class {class_idx} ({data_loader.classes[class_idx]}): "
                  f"{orig_count} original → generating {needed} synthetic...")

            noise  = np.random.normal(0, 1, (needed, self.latent_dim))
            labels = np.full((needed, 1), class_idx)

            imgs = self.generator.predict([noise, labels], verbose=0)
            generated_images.append(imgs)
            generated_labels.extend([class_idx] * needed)

        X_synthetic = np.vstack(generated_images)
        y_synthetic = np.array(generated_labels)

        print(f"\nTotal synthetic images generated: {len(X_synthetic)}")
        print(f"Expected combined total: {len(X_synthetic) + sum(class_counts.values())}")
        return X_synthetic, y_synthetic

In [ ]:
# Initialize and train ACGAN
acgan = ACGAN(img_shape=(128, 128, 1), num_classes=4, latent_dim=100)

In [ ]:
print("\nGenerator Summary:")
acgan.generator.summary()

In [ ]:
print("\nDiscriminator Summary:")
acgan.discriminator.summary()

In [ ]:
history = acgan.train(X_original, y_original, epochs=100, batch_size=32, save_interval=10)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['d_loss'], label='Discriminator Loss', alpha=0.7)
plt.plot(history['g_loss'], label='Generator Loss', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('ACGAN Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('acgan_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Original counts from loaded data
original_class_counts = {
    0: int(np.sum(y_original == 0)),   # Cyst:   3709
    1: int(np.sum(y_original == 1)),   # Normal: 5077
    2: int(np.sum(y_original == 2)),   # Stone:  1377
    3: int(np.sum(y_original == 3)),   # Tumor:  2283
}
print("\nOriginal class counts:", original_class_counts)

In [ ]:
X_synthetic, y_synthetic = acgan.generate_images_topup(
    class_counts=original_class_counts,
    target_per_class=7000
)

In [ ]:
print(f"\nSynthetic dataset shape: {X_synthetic.shape}")
print(f"Synthetic labels shape:  {y_synthetic.shape}")

============================================================================
SECTION 8: VISUALIZE SYNTHETIC IMAGES
============================================================================

In [ ]:
def visualize_synthetic_samples(generator, classes, num_samples=4):
    fig, axes = plt.subplots(len(classes), num_samples, figsize=(12, 10))

    for class_idx, class_name in enumerate(classes):
        noise  = np.random.normal(0, 1, (num_samples, 100))
        labels = np.full((num_samples, 1), class_idx)

        gen_imgs = generator.predict([noise, labels], verbose=0)
        gen_imgs = (gen_imgs + 1) / 2.0 

        for i in range(num_samples):
            axes[class_idx, i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
            axes[class_idx, i].axis('off')
            if i == 0:
                axes[class_idx, i].set_title(class_name, fontsize=12)

    plt.tight_layout()
    plt.savefig('synthetic_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
visualize_synthetic_samples(acgan.generator, data_loader.classes)

============================================================================
SECTION 9: CREATE BALANCED COMBINED DATASET
============================================================================

In [ ]:
X_combined = np.vstack([X_original, X_synthetic])
y_combined = np.concatenate([y_original, y_synthetic])

In [ ]:
print("\n" + "="*60)
print("COMBINED DATASET")
print("="*60)
print(f"Total samples: {len(X_combined)}")   # Expected: 28,000
print(f"Shape: {X_combined.shape}")

In [ ]:
# Verify per-class counts
unique, counts = np.unique(y_combined, return_counts=True)
print("\nCombined Class Distribution:")
for idx, count in zip(unique, counts):
    print(f"  {data_loader.classes[idx]}: {count}")  # Each should be 7,000

In [ ]:
visualize_class_distribution(y_combined, data_loader.classes,
                             "Combined Dataset (Original + Synthetic) — 28,000 images")
visualize_tsne_pca(X_combined, y_combined, data_loader.classes, "Combined Dataset")

============================================================================
SECTION 9.5: QUANTITATIVE REALISM METRICS (FID & SSIM)
============================================================================

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm
from skimage.metrics import structural_similarity as ssim

In [ ]:
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

In [ ]:
def scale_images_for_inception(images):
    """Convert (N,128,128,1) [-1,1] → (N,299,299,3) InceptionV3-preprocessed"""
    images = (images + 1) * 127.5                                   # [-1,1] → [0,255]
    images = tf.image.grayscale_to_rgb(
        tf.constant(images.astype(np.uint8), dtype=tf.float32))     # 1ch → 3ch
    images = tf.image.resize(images, (299, 299)).numpy()
    return images

In [ ]:
def calculate_fid(model, images1, images2):
    img1 = preprocess_input(scale_images_for_inception(images1))
    img2 = preprocess_input(scale_images_for_inception(images2))

    act1 = model.predict(img1, verbose=0)
    act2 = model.predict(img2, verbose=0)

    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    ssdiff  = np.sum((mu1 - mu2) ** 2)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [ ]:
def calculate_ssim(images1, images2, data_range=2.0):
    sample_size = min(len(images1), len(images2), 1000)
    idx1 = np.random.choice(len(images1), sample_size, replace=False)
    idx2 = np.random.choice(len(images2), sample_size, replace=False)

    ssim_scores = [
        ssim(images1[idx1[i], :, :, 0], images2[idx2[i], :, :, 0], data_range=data_range)
        for i in range(sample_size)
    ]
    return np.mean(ssim_scores)

In [ ]:
print("\n" + "="*60)
print("CALCULATING REALISM METRICS")
print("="*60)

In [ ]:
sample_size_fid = min(len(X_original), len(X_synthetic))
print(f"\nCalculating FID using {sample_size_fid} real and {sample_size_fid} synthetic images...")

In [ ]:
fid_score = calculate_fid(
    inception_model,
    X_original[np.random.choice(len(X_original), sample_size_fid, replace=False)],
    X_synthetic[np.random.choice(len(X_synthetic), sample_size_fid, replace=False)]
)
print(f"Fréchet Inception Distance (FID): {fid_score:.4f}")

In [ ]:
print("\nCalculating SSIM between sampled real and synthetic images...")
ssim_score = calculate_ssim(X_original, X_synthetic)
print(f"Average Structural Similarity Index (SSIM): {ssim_score:.4f}")

============================================================================
SECTION 10: TRANSFER LEARNING MODELS
============================================================================

In [ ]:
def build_transfer_model(base_model_name, input_shape, num_classes):
    """Build transfer learning model with correct preprocessing per architecture"""
    input_layer = keras.Input(shape=input_shape)

    # Convert grayscale → 3-channel for all models
    x = layers.Concatenate()([input_layer, input_layer, input_layer])

    if base_model_name == 'EfficientNetV2':
        x = layers.Rescaling(scale=127.5, offset=127.5)(x)
        base_model = EfficientNetV2B0(weights='imagenet', include_top=False,
                                      input_shape=(128, 128, 3))
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False,
                           input_shape=(128, 128, 3))
    elif base_model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False,
                                 input_shape=(128, 128, 3))
    else:
        raise ValueError(f"Unknown model: {base_model_name}")

    # Phase 1: freeze entire base model for warm-up
    base_model.trainable = False

    x      = base_model(x)
    x      = layers.GlobalAveragePooling2D()(x)
    x      = layers.Dense(256, activation='relu')(x)
    x      = layers.Dropout(0.5)(x)
    x      = layers.Dense(128, activation='relu')(x)
    x      = layers.Dropout(0.3)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=output, name=base_model_name)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model, base_model

In [ ]:
def train_with_finetuning(model, base_model, X_train, y_train, X_val, y_val,
                          model_name, warmup_epochs=10, finetune_epochs=20):
    """
    Two-phase training:
      Phase 1 (warm-up): train only the custom head with base frozen
      Phase 2 (fine-tune): unfreeze last 20 layers, train at lower LR
    Paper: "fine-tuned independently on both original and ACGAN-augmented dataset"
    """
    print(f"\n{'='*60}")
    print(f"TRAINING {model_name}")
    print(f"{'='*60}")

    callbacks_warmup = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
        ModelCheckpoint(f'{model_name}_warmup_best.keras', monitor='val_accuracy',
                        save_best_only=True, mode='max')
    ]

    print(f"\nPhase 1: Warm-up ({warmup_epochs} epochs, base model frozen)")
    history_warmup = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=warmup_epochs,
        batch_size=32,
        callbacks=callbacks_warmup,
        verbose=1
    )

    # Phase 2: Unfreeze last 20 layers of base model for fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False  # keep all layers except last 20 frozen

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks_finetune = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8),
        ModelCheckpoint(f'{model_name}_best.keras', monitor='val_accuracy',
                        save_best_only=True, mode='max')
    ]

    print(f"\nPhase 2: Fine-tuning ({finetune_epochs} epochs, last 20 layers unfrozen, LR=1e-5)")
    history_finetune = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=finetune_epochs,
        batch_size=32,
        callbacks=callbacks_finetune,
        verbose=1
    )

    # Merge histories for plotting
    combined_history = {
        'accuracy':     history_warmup.history['accuracy']     + history_finetune.history['accuracy'],
        'val_accuracy': history_warmup.history['val_accuracy'] + history_finetune.history['val_accuracy'],
        'loss':         history_warmup.history['loss']         + history_finetune.history['loss'],
        'val_loss':     history_warmup.history['val_loss']     + history_finetune.history['val_loss'],
    }
    return combined_history

============================================================================
SECTION 11: DATA SPLITTING
============================================================================

In [ ]:
print("\n" + "="*70)
print("DATA SPLITTING — 70% Train / 15% Val / 15% Test (Stratified)")
print("="*70)

In [ ]:
# Step 1: split off 30% for val + test
X_train_orig, X_temp, y_train_orig, y_temp = train_test_split(
    X_original, y_original,
    test_size=0.30, random_state=42, stratify=y_original
)
# Step 2: split the 30% equally into 15% val and 15% test
X_val_orig, X_test_orig, y_val_orig, y_test_orig = train_test_split(
    X_temp, y_temp,
    test_size=0.50, random_state=42, stratify=y_temp
)

In [ ]:
print(f"\nOriginal data splits:")
print(f"  Train: {X_train_orig.shape}  ({len(X_train_orig)/len(X_original)*100:.1f}%)")
print(f"  Val:   {X_val_orig.shape}   ({len(X_val_orig)/len(X_original)*100:.1f}%)")
print(f"  Test:  {X_test_orig.shape}  ({len(X_test_orig)/len(X_original)*100:.1f}%)")

============================================================================
SECTION 11: TRAIN ON ORIGINAL DATASET
============================================================================

In [ ]:
print("\n" + "="*70)
print("TRAINING ON ORIGINAL DATASET")
print("="*70)

In [ ]:
models_original    = {}
histories_original = {}

In [ ]:
for model_name in ['VGG16', 'MobileNetV2', 'EfficientNetV2']:
    model, base_model = build_transfer_model(model_name, (128, 128, 1), 4)
    history = train_with_finetuning(
        model, base_model,
        X_train_orig, y_train_orig,
        X_val_orig, y_val_orig,
        f'{model_name}_original'
    )
    models_original[model_name]    = model
    histories_original[model_name] = history

============================================================================
SECTION 12: TRAIN ON AUGMENTED DATASET
============================================================================

In [ ]:
print("\n" + "="*70)
print("TRAINING ON AUGMENTED DATASET")
print("="*70)

Apply the same 70/15/15 split to the combined dataset —
but only the TRAINING portion uses synthetic images.
Val and test are identical to original splits (untouched).

In [ ]:
# Build augmented training set: original train + ALL synthetic
X_train_aug = np.vstack([X_train_orig, X_synthetic])
y_train_aug  = np.concatenate([y_train_orig, y_synthetic])

In [ ]:
# Shuffle
shuffle_idx  = np.random.permutation(len(X_train_aug))
X_train_aug  = X_train_aug[shuffle_idx]
y_train_aug  = y_train_aug[shuffle_idx]

In [ ]:
print(f"\nAugmented training set: {X_train_aug.shape}")
print(f"Validation set (original only): {X_val_orig.shape}")
print(f"Test set (original only):       {X_test_orig.shape}")

In [ ]:
models_augmented    = {}
histories_augmented = {}

In [ ]:
for model_name in ['VGG16', 'MobileNetV2', 'EfficientNetV2']:
    model, base_model = build_transfer_model(model_name, (128, 128, 1), 4)
    history = train_with_finetuning(
        model, base_model,
        X_train_aug, y_train_aug,
        X_val_orig, y_val_orig,
        f'{model_name}_augmented'
    )
    models_augmented[model_name]    = model
    histories_augmented[model_name] = history

============================================================================
SECTION 13: EVALUATION - Val + Test sets
============================================================================

In [ ]:
from sklearn.metrics import (classification_report, confusion_matrix,
                              accuracy_score, roc_auc_score, roc_curve)

In [ ]:
def plot_training_history(history, title_prefix):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'],     label='train_acc')
    plt.plot(history['val_accuracy'], label='val_acc')
    plt.title(f'{title_prefix} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(history['loss'],     label='train_loss')
    plt.plot(history['val_loss'], label='val_loss')
    plt.title(f'{title_prefix} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{title_prefix.replace(" ", "_").lower()}_training_plot.png',
                dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def evaluate_model(model, X_eval, y_eval, classes, model_name, split_name='val'):
    print(f"\nEvaluating {model_name} on {split_name} set...")

    y_pred_prob = model.predict(X_eval, verbose=0)
    y_pred      = np.argmax(y_pred_prob, axis=1)

    acc = accuracy_score(y_eval, y_pred)
    print(f"Accuracy: {acc:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_eval, y_pred, target_names=classes))

    # AUC-ROC (one-vs-rest, per class + macro)
    try:
        auc_macro = roc_auc_score(
            tf.keras.utils.to_categorical(y_eval, num_classes=len(classes)),
            y_pred_prob,
            multi_class='ovr', average='macro'
        )
        print(f"Macro AUC-ROC: {auc_macro:.4f}")
    except Exception as e:
        print(f"AUC-ROC could not be computed: {e}")
        auc_macro = None

    # Confusion matrix
    cm = confusion_matrix(y_eval, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix — {model_name} ({split_name})')
    plt.savefig(f'confusion_matrix_{model_name}_{split_name}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

    return {'accuracy': acc, 'auc_macro': auc_macro, 'confusion_matrix': cm}

In [ ]:
# Evaluate all models on validation AND test sets
evaluation_results = {'original': {}, 'augmented': {}}

In [ ]:
for model_name in ['VGG16', 'MobileNetV2', 'EfficientNetV2']:
    # Original models
    model_orig = models_original[model_name]
    hist_orig  = histories_original[model_name]
    plot_training_history(hist_orig, f'{model_name} (Original)')

    res_val  = evaluate_model(model_orig, X_val_orig,  y_val_orig,
                              data_loader.classes, model_name, 'val_original')
    res_test = evaluate_model(model_orig, X_test_orig, y_test_orig,
                              data_loader.classes, model_name, 'test_original')
    evaluation_results['original'][model_name] = {
        'val': res_val, 'test': res_test
    }
    model_orig.save(f'{model_name}_original_model.keras')
    print(f"Saved: {model_name}_original_model.keras")

    # Augmented models
    model_aug = models_augmented[model_name]
    hist_aug  = histories_augmented[model_name]
    plot_training_history(hist_aug, f'{model_name} (Augmented)')

    res_val  = evaluate_model(model_aug, X_val_orig,  y_val_orig,
                              data_loader.classes, model_name, 'val_augmented')
    res_test = evaluate_model(model_aug, X_test_orig, y_test_orig,
                              data_loader.classes, model_name, 'test_augmented')
    evaluation_results['augmented'][model_name] = {
        'val': res_val, 'test': res_test
    }
    model_aug.save(f'{model_name}_augmented_model.keras')
    print(f"Saved: {model_name}_augmented_model.keras")

============================================================================
SECTION 14: COMPARISON SUMMARY TABLE
============================================================================

In [ ]:
summary_rows = []
for model_name in ['VGG16', 'MobileNetV2', 'EfficientNetV2']:
    orig_val_acc  = evaluation_results['original'][model_name]['val']['accuracy']
    orig_test_acc = evaluation_results['original'][model_name]['test']['accuracy']
    aug_val_acc   = evaluation_results['augmented'][model_name]['val']['accuracy']
    aug_test_acc  = evaluation_results['augmented'][model_name]['test']['accuracy']

    summary_rows.append({
        'Model':                    model_name,
        'Original Val Acc':         f"{orig_val_acc:.4f}",
        'Original Test Acc':        f"{orig_test_acc:.4f}",
        'Augmented Val Acc':        f"{aug_val_acc:.4f}",
        'Augmented Test Acc':       f"{aug_test_acc:.4f}",
        'Paper Original Target':    {'VGG16': '99.2%', 'MobileNetV2': '97.2%', 'EfficientNetV2': '97.62%'}[model_name],
        'Paper Augmented Target':   {'VGG16': '97.3%', 'MobileNetV2': '95.1%', 'EfficientNetV2': '97.0%'}[model_name],
    })

In [ ]:
summary_df = pd.DataFrame(summary_rows)
print('\n' + '='*70)
print('MODEL COMPARISON SUMMARY vs. PAPER TARGETS')
print('='*70)
print(summary_df.to_string(index=False))
summary_df.to_csv('model_comparison_summary.csv', index=False)
print('\nSaved: model_comparison_summary.csv')

============================================================================
SECTION 15: CLEANUP
============================================================================

In [ ]:
print('\nAll done. Output files:')
print('  - class_distribution_original.png')
print('  - tsne_pca_*.png')
print('  - acgan_training_history.png')
print('  - synthetic_samples.png')
print('  - confusion_matrix_*.png')
print('  - *_training_plot.png')
print('  - *.keras  (saved models)')
print('  - model_comparison_summary.csv')